In [ ]:
import pandas as pd
import scanpy as sc
import numpy as np
import scipy
import matplotlib.pyplot as plt
import rpy2
from matplotlib import rcParams # for figure size adjustment
import os

# Load data

In [ ]:
data_path = "../../OneDrive/Documents/scData/hiv/processed_data/"


def load_data (file_name):
    tdata = sc.read_10x_h5(data_path + file_name)
    
    # Some variables can be the same name, which could make errors in the downstream analysis.
    # Add numbers (1, 2, 3, ...) to the variables to make the variables unique.
    tdata.var_names_make_unique()
    
    print(f"The shape of the dataset: n_obs {tdata.shape[0]} x n_vars {tdata.shape[1]}.")
    print("Init barcode count:", tdata.shape[0])
    print("Init_gene_count:", tdata.shape[1], "\n")
    return tdata
    
normal_1 = load_data("normal_1_filtered_feature_bc_matrix.h5")
normal_2 = load_data("normal_2_filtered_feature_bc_matrix.h5")
normal_3 = load_data("normal_3_filtered_feature_bc_matrix.h5")
hiv_1 = load_data("hiv_1_filtered_feature_bc_matrix.h5")
hiv_2 = load_data("hiv_2_filtered_feature_bc_matrix.h5")
hiv_3 = load_data("hiv_3_filtered_feature_bc_matrix.h5")

In [ ]:
print(normal_1, "\n")
print(normal_2, "\n")
print(normal_3, "\n")
print(hiv_1, "\n")
print(hiv_2, "\n")
print(hiv_3, "\n")

# Quality control

## Basic cell filtering

In [ ]:
for adata in [normal_1, normal_2, normal_3, hiv_1, hiv_2, hiv_3]:
    sc.pp.filter_cells(adata, min_genes = 200)
    sc.pp.filter_genes(adata, min_cells = 3)

## Draw QC plots

In [ ]:
# annotate genes to adata
for adata in [normal_1, normal_2, normal_3, hiv_1, hiv_2, hiv_3]:
    # mitochondrial genes
    adata.var["MT"] = adata.var_names.str.startswith("MT-")
    # ribosomal genes
    adata.var["ribo"] = adata.var_names.str.startswith("RPS") | adata.var_names.str.startswith("RPL")
    # hemoglobin genes.
    # adata.var["hb"] = adata.var_names.str.contains("^HB[^(p)]")
    # Malat1
    adata.var["MALAT1"] = adata.var_names.str.contains("MALAT1")
    
    sc.pp.calculate_qc_metrics(
        adata,
        qc_vars = ["MT", "ribo", "MALAT1"],
        inplace = True,
        percent_top = [20],
        log1p = True
    )


In [ ]:
# Highest expressed gene plots

for adata in [normal_1, normal_2, normal_3, hiv_1, hiv_2, hiv_3]:
    sc.pl.highest_expr_genes(adata, n_top = 20)
    # MALAT1 frequently detected in RNA-seq data captured by poly-A
    # A technical bias need to remove
    # RPS, RPL: ribosomal genes
    # MT-: mitochondrial genes

In [ ]:
# Histogram of the cell quality

for adata in [normal_1, normal_2, normal_3, hiv_1, hiv_2, hiv_3]:
    fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(15, 4))
    n_bins = 50
    ax[0].hist(adata.obs.n_genes_by_counts, bins = n_bins)
    ax[0].set_title("n_genes_by_counts")
    ax[1].hist(adata.obs.total_counts, bins = n_bins)
    ax[1].set_title("total_counts")
    # ax[2].hist(adata.obs.pct_counts_Malat1, bins=50)
    # ax[2].set_title("pct_counts_MALAT1")
    ax[3].hist(adata.obs.pct_counts_ribo, bins = n_bins)
    ax[3].set_title("pct_counts_ribo")
    # ax[1].hist(adata.obs.pct_counts_hb, bins=50)
    # ax[1].set_title("pct_counts_hb")
    ax[2].hist(adata.obs.pct_counts_MT, bins = n_bins)
    ax[2].set_title("pct_counts_MT")

    plt.tight_layout()
    plt.show

In [ ]:
# violin plot of the cell quality

for adata in [normal_1, normal_2, normal_3, hiv_1, hiv_2, hiv_3]:
    sc.pl.violin(
        adata,
        ["n_genes_by_counts", "total_counts", "pct_counts_MT"],
        jitter=0.4,
        multi_panel=True
    )

In [ ]:
# scatterplot of the cell quality

rcParams['figure.figsize'] = 5, 5 # width, height

for adata in [normal_1, normal_2, normal_3, hiv_1, hiv_2, hiv_3]:
    # plt.figure(figsize=(5, 5))
    sc.pl.scatter(
        adata,
        "total_counts",
        "n_genes_by_counts",
        color="pct_counts_MT"
    )
    # plt.show()
 

## MAD filtering

In [ ]:
# Calculate MAD
# https://tomorrow-lab.github.io/posts/drylab/python_scRNA_QC.html

# Calculate MAD
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * scipy.stats.median_abs_deviation(M)) | (
        np.median(M) + nmads * scipy.stats.median_abs_deviation(M) < M
    )
    return outlier

# Check the outliers
for adata in [normal_1, normal_2, normal_3, hiv_1, hiv_2, hiv_3]:
    adata.obs["outlier"] = (
        is_outlier(adata, "log1p_total_counts", 5)
        | is_outlier(adata, "log1p_n_genes_by_counts", 5)
        | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
    )
    print(adata.obs.outlier.value_counts(), "\n")

# Check the outliers for MT
for adata in [normal_1, normal_2, normal_3, hiv_1, hiv_2, hiv_3]:
    adata.obs["MT_outlier"] = is_outlier(adata, "pct_counts_MT", 3) |(
        adata.obs["pct_counts_MT"] > 8
    )
    print(adata.obs.MT_outlier.value_counts(), "\n")

## Actual filtering using MAD

In [ ]:
# Filter out the outliers
def MAD_filtering (adata):
    print(f"Cell number before filtering: {adata.n_obs}")
    adata = adata[(~adata.obs.outlier) & (~adata.obs.MT_outlier)]
    
    print(f"Cell number after filtering: {adata.n_obs}")
    print()
    return adata

normal_1 = MAD_filtering(normal_1)
normal_2 = MAD_filtering(normal_2)
normal_3 = MAD_filtering(normal_3)
hiv_1 = MAD_filtering(hiv_1)
hiv_2 = MAD_filtering(hiv_2)
hiv_3 = MAD_filtering(hiv_3)

# Transient clustering for SoupX

In [ ]:
from scipy.sparse import csc_matrix, save_npz

def preprocessing_data_to_r (adata, annotation):
    adata_pp = adata.copy() # copy adata to other variant
    sc.pp.normalize_per_cell(adata_pp)  # normalize RNA count
    sc.pp.log1p(adata_pp)   # calculate log1p
    sc.pp.pca(adata_pp) # PCA calculation
    sc.pp.neighbors(adata_pp) # neighboring
    sc.tl.leiden(adata_pp, key_added="soupx_groups") # leiden clustering

    # 클러스터를 .obs에 soupx_groups로 추가하고 벡터로 저장합니다.
    soupx_groups = adata_pp.obs["soupx_groups"]

    # AnnData 객체의 복사본을 삭제해 메모리를 절약합니다.
    del adata_pp

    # 세포, 유전자, 필터링된 카운터 행렬을 저장합니다.
    # SoupX에는 모양 특징 x 바코드 행렬이 필요하므로 transpose해야 합니다.
    cells = adata.obs_names
    genes = adata.var_names
    data = adata.X.T
    
    # Save data in "./data_to_r/" folder for further analysis
    # 경로 설정
    directory = '../file_to_r/'

    # 경로가 존재하지 않으면 생성
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    # 5. 파일명 생성
    filename_cells = f"{annotation}_cells.csv"
    filepath_cells = os.path.join(directory, filename_cells)
    filename_genes = f"{annotation}_genes.csv"
    filepath_genes = os.path.join(directory, filename_genes)
    filename_data = f"{annotation}_data.npz"
    filepath_data = os.path.join(directory, filename_data)

    # 6. Feather 파일로 저장
    cells.to_series().to_csv(filepath_cells, index=False, header=['Index'])
    genes.to_series().to_csv(filepath_genes, index=False, header=['Index'])
    save_npz(filepath_data, data)

    print(f"DataFrame is saved as {filepath_cells}")
    print(f"DataFrame is saved as {filepath_genes}")
    print(f"DataFrame is saved as {filepath_data}")

In [ ]:
preprocessing_data_to_r(normal_1, "normal_1")

In [ ]:
# save raw_feature_bc_matrix.h5 for further analysis in r
data_path = "../../OneDrive/Documents/scData/hiv/raw_data/"

def load_data (file_name):
    tdata = sc.read_10x_h5(data_path + file_name)
    
    tdata.var_names_make_unique()
    
    # print(f"The shape of the dataset: n_obs {tdata.shape[0]} x n_vars {tdata.shape[1]}.")
    # print("Init barcode count:", tdata.shape[0])
    # print("Init_gene_count:", tdata.shape[1], "\n")
    return tdata.X.T
    
normal_1_tod = load_data("normal_1_raw_feature_bc_matrix.h5")
normal_2_tod = load_data("normal_2_raw_feature_bc_matrix.h5")
normal_3_tod = load_data("normal_3_raw_feature_bc_matrix.h5")

In [ ]:
type(normal_1_tod)

# Doublet removing using scDblFinder

In [ ]:
# for run R code in jupyter notebook
import os
os.environ['R_HOME'] = "C:/Users/heung/anaconda3/Lib/R"

In [ ]:
# to use R in jupyter notebook
import anndata2ri
import logging

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro
from rpy2.robjects.packages import importr, PackageNotInstalledError

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate

%load_ext rpy2.ipython

In [ ]:
normal_1_t = anndata2ri.py2rpy(normal_1.X.T)
normal_2_t = anndata2ri.py2rpy(normal_2.X.T)
normal_3_t = anndata2ri.py2rpy(normal_3.X.T)
hiv_1_t = anndata2ri.py2rpy(hiv_1.X.T)
hiv_2_t = anndata2ri.py2rpy(hiv_2.X.T)
hiv_3_t = anndata2ri.py2rpy(hiv_3.X.T)

In [ ]:
%%R -i normal_1_t,normal_2_t,normal_3_t,hiv_1_t,hiv_2_t,hiv_3_t

saveRDS(normal_1_t, "../file_to_r/normal_1_t.rds")
saveRDS(normal_2_t, "../file_to_r/normal_2_t.rds")
saveRDS(normal_3_t, "../file_to_r/normal_3_t.rds")
saveRDS(hiv_1_t, "../file_to_r/hiv_1_t.rds")
saveRDS(hiv_2_t, "../file_to_r/hiv_2_t.rds")
saveRDS(hiv_3_t, "../file_to_r/hiv_3_t.rds")

여기서 R로 갔다가 scDblFinder 돌리고 다시 돌아옴

In [ ]:
%%R -o normal_1_dbl_score -o normal_1_dbl_class -o normal_2_dbl_score -o normal_2_dbl_class -o normal_3_dbl_score -o normal_3_dbl_class -o hiv_1_dbl_score -o hiv_1_dbl_class -o hiv_2_dbl_score -o hiv_2_dbl_class -o hiv_3_dbl_score -o hiv_3_dbl_class

# Load libraries
normal_1_dbl_score <- readRDS("../file_to_py/normal_1_dbl_score.rds")
normal_1_dbl_class <- readRDS("../file_to_py/normal_1_dbl_class.rds")
normal_2_dbl_score <- readRDS("../file_to_py/normal_2_dbl_score.rds")
normal_2_dbl_class <- readRDS("../file_to_py/normal_2_dbl_class.rds")
normal_3_dbl_score <- readRDS("../file_to_py/normal_3_dbl_score.rds")
normal_3_dbl_class <- readRDS("../file_to_py/normal_3_dbl_class.rds")

hiv_1_dbl_score <- readRDS("../file_to_py/hiv_1_dbl_score.rds")
hiv_1_dbl_class <- readRDS("../file_to_py/hiv_1_dbl_class.rds")
hiv_2_dbl_score <- readRDS("../file_to_py/hiv_2_dbl_score.rds")
hiv_2_dbl_class <- readRDS("../file_to_py/hiv_2_dbl_class.rds")
hiv_3_dbl_score <- readRDS("../file_to_py/hiv_3_dbl_score.rds")
hiv_3_dbl_class <- readRDS("../file_to_py/hiv_3_dbl_class.rds")

In [ ]:
# Save the result as adata.obs
normal_1.obs["scDblFinder_score"] = normal_1_dbl_score
normal_1.obs["scDblFinder_class"] = normal_1_dbl_class
normal_2.obs["scDblFinder_score"] = normal_2_dbl_score
normal_2.obs["scDblFinder_class"] = normal_2_dbl_class
normal_3.obs["scDblFinder_score"] = normal_3_dbl_score
normal_3.obs["scDblFinder_class"] = normal_3_dbl_class

hiv_1.obs["scDblFinder_score"] = hiv_1_dbl_score
hiv_1.obs["scDblFinder_class"] = hiv_1_dbl_class
hiv_2.obs["scDblFinder_score"] = hiv_2_dbl_score
hiv_2.obs["scDblFinder_class"] = hiv_2_dbl_class
hiv_3.obs["scDblFinder_score"] = hiv_3_dbl_score
hiv_3.obs["scDblFinder_class"] = hiv_3_dbl_class

In [ ]:
print(type(normal_1.obs.scDblFinder_score))


print(type(hiv_1.obs))
print(type(hiv_1.obs.scDblFinder_score))

In [ ]:
print(normal_1.obs.scDblFinder_class.value_counts(), "\n")
print(normal_2.obs.scDblFinder_class.value_counts(), "\n")
print(normal_3.obs.scDblFinder_class.value_counts(), "\n")
print(hiv_1.obs.scDblFinder_class.value_counts(), "\n")
print(hiv_2.obs.scDblFinder_class.value_counts(), "\n")
print(hiv_3.obs.scDblFinder_class.value_counts(), "\n")

In [ ]:
# Doublet score histogram

plt.figure()
plt.hist(normal_1.obs.scDblFinder_score, bins=50)
plt.title("normal_1 doublet score")
plt.show()

plt.figure()
plt.hist(normal_2.obs.scDblFinder_score, bins=50)
plt.title("normal_2 doublet score")
plt.show()

plt.figure()
plt.hist(normal_3.obs.scDblFinder_score, bins=50)
plt.title("normal_3 doublet score")
plt.show()

plt.figure()
plt.hist(hiv_1.obs.scDblFinder_score, bins=50)
plt.title("hiv_1 doublet score")
plt.show()

plt.figure()
plt.hist(hiv_2.obs.scDblFinder_score, bins=50)
plt.title("hiv_2 doublet score")
plt.show()

plt.figure()
plt.hist(hiv_3.obs.scDblFinder_score, bins=50)
plt.title("hiv_3 doublet score")
plt.show()

In [ ]:
# save quality controled files

normal_1.write("../qc_files/normal_1_quality_control.h5ad")
normal_2.write("../qc_files/normal_2_quality_control.h5ad")
normal_3.write("../qc_files/normal_3_quality_control.h5ad")
hiv_1.write("../qc_files/hiv_1_quality_control.h5ad")
hiv_2.write("../qc_files/hiv_2_quality_control.h5ad")
hiv_3.write("../qc_files/hiv_3_quality_control.h5ad")